In [103]:
#预测一部电影的评分，属于分类问题。输入一段具体的文本，输出具体的评分
#需要完成：
#1.文本预处理，分词，停用词过滤、低频次过滤、特殊符号的过滤
#文本转化为向量，三种方式，tf—idf,word2vec以及BERT
#训练逻辑回归和朴素贝叶斯模型，做交叉验证
#评估模型的正确率


In [104]:
#导入数据处理基础包
import numpy as np
import pandas as pd

#导入用于计数的包
from collections import Counter

#导入tf_idf相关的包
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#导入模型评估的包
from sklearn import metrics

#导入与word2vec相关的包
from gensim.models import KeyedVectors

#导入与bert embedding相关的包
# from m bert_embedding import BertEmbedding
# import mxnet

#tqdm包是用来对可迭代对执行时生成一个进度条用以监视程序运行过程
from tqdm import tqdm

#导入其他功能包，爬虫
import requests
import os

In [105]:
#读取数据
data = pd.read_csv('E:\Game\pythonProject\douban\DMSC.csv')

#观察数据格式
print(data.head())

#输出数据的一些相关信息
data.info()

   ID           Movie_Name_EN Movie_Name_CN  Crawl_Date  Number Username  \
0   0  Avengers Age of Ultron        复仇者联盟2  2017-01-22       1       然潘   
1  10  Avengers Age of Ultron        复仇者联盟2  2017-01-22      11       影志   
2  20  Avengers Age of Ultron        复仇者联盟2  2017-01-22      21     随时流感   
3  30  Avengers Age of Ultron        复仇者联盟2  2017-01-22      31     乌鸦火堂   
4  40  Avengers Age of Ultron        复仇者联盟2  2017-01-22      41    办公室甜心   

         Date  Star                                            Comment  Like  
0  2015-05-13     3                                      连奥创都知道整容要去韩国。  2404  
1  2015-04-30     4   “一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...   381  
2  2015-04-28     2                                 奥创弱爆了弱爆了弱爆了啊！！！！！！   120  
3  2015-05-08     4   与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...    30  
4  2015-05-10     5   看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...    16  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212506 entries, 0 t

In [106]:
#只保留数据中需要的两列：Comment列和Star列
data = data[['Comment','Star']]

#观察新的数据的格式
data.head()


,Comment,Star
0,连奥创都知道整容要去韩国。,3
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,4
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,2
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,4
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,5


In [107]:
#star代表具体的评分，这个项目中，预测正面还是负面，1.2是负面，345是正面
data['Star']=(data.Star/3).astype(int)
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1


In [108]:
#取前100数据
data = data[:100]
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1


In [109]:
#统计star等于1的数量
Counter(data['Star']==1)

Counter({True: 78, False: 22})

In [110]:
#去掉依稀而无用的字符，自定义一个几何字符，从文中去掉
#DataFrame.apply(func, axis=0, broadcast=False, raw=False, reduce=None, args=(), **kwds)
#lambda:匿名函数。
data['Comment']=data['Comment'].apply(lambda x:x.replace('，','').replace('。',''))

In [111]:
#输出进度条..desc=自定义名字
tqdm.pandas(desc='Untitled-1')
data['Comment']=data['Comment'].apply(lambda x:x.replace('，','').replace('。',''))

In [112]:
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国,1
1,“一个没有黑暗面的人不值得信任” 第二部剥去冗长的铺垫开场即高潮、一直到结束会有人觉得只剩...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战...,1
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹...,1


In [113]:
#导入中文分词包jieba，用jieba对原始文本做分词
import jieba
def comment_cut(content):
    return list(jieba.cut(content.strip()))
data['coment_processed']=data['Comment'].progress_apply(comment_cut)

Untitled-1: 100%|██████████| 100/100 [00:00<00:00, 324.49it/s]


In [114]:
data.head()

,Comment,Star,coment_processed
0,连奥创都知道整容要去韩国,1,"[连, 奥创, 都, 知道, 整容, 要, 去, 韩国]"
1,“一个没有黑暗面的人不值得信任” 第二部剥去冗长的铺垫开场即高潮、一直到结束会有人觉得只剩...,1,"[“, 一个, 没有, 黑暗面, 的, 人, 不, 值得, 信任, ”, , 第二部, 剥..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,"[奥创, 弱, 爆, 了, 弱, 爆, 了, 弱, 爆, 了, 啊, ！, ！, ！, ！,..."
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战...,1,"[与, 第一集, 不同, 承上启下, 阴郁, 严肃, 但, 也, 不会, 不, 好看, 啊,..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹...,1,"[看毕, 我, 激动, 地, 对, 友人, 说, 等等, 奥创, 要, 来, 毁灭, 台北,..."


In [115]:
#设定停用词并从文本中去掉停用词
if not os.path.exists('E:\Game\pythonProject\douban\stopWord.json'):
    stopWord = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt")
    with open("data/stopWord.json","wb") as f:
        f.write(stopWord.content)

#读取停用词表，并保存在列表中
with open("E:\Game\pythonProject\douban\stopWord.json","r",encoding='utf-8') as f:
    stopWords=f.read().split("\n")

#去除停用词
def rm_stop_word(wordList):
    return[word for word in wordList if word not in stopWords]
#process_apply（）函数的作用等同于.apply（）的作用，只是写成process_apply（）的形势次啊能被tqdm监控
data['coment_processed'] = data['coment_processed'].progress_apply(rm_stop_word)

Untitled-1: 100%|██████████| 100/100 [00:00<00:00, 4177.51it/s]


In [116]:
data.head()


,Comment,Star,coment_processed
0,连奥创都知道整容要去韩国,1,"[奥创, 知道, 整容, 韩国]"
1,“一个没有黑暗面的人不值得信任” 第二部剥去冗长的铺垫开场即高潮、一直到结束会有人觉得只剩...,1,"[一个, 没有, 黑暗面, 值得, 信任, , 第二部, 剥去, 冗长, 铺垫, 开场, ..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战...,1,"[第一集, 不同, 承上启下, 阴郁, 严肃, 不会, 好看, 本来, 喜欢, 漫威, 电影..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹...,1,"[看毕, 激动, 友人, 说, 奥创, 毁灭, 台北, 厚, 拍了拍, 肩膀, 没事, 反正..."


In [117]:
#去除低频词，去掉词频小于10的单词，斌吧结果保存到data['comment_processed]l里
from os import remove
wordCount = {}
for c in data['coment_processed']:
    for w in c:
        if w in wordCount:
            wordCount[w] += 1
        else:
            wordCount[w] = 1
def remove_low_freqen_word(c):
    return[w for w in c if wordCount[w]>=2]
data['coment_processed'] = data['coment_processed'].progress_apply(remove_low_freqen_word)


Untitled-1: 100%|██████████| 100/100 [00:00<00:00, 50051.36it/s]


In [118]:
#按词频排序
sorted(wordCount.items(),key=lambda x:x[1],reverse=True)

[(' ', 91),
 ('…', 18),
 ('没有', 14),
 ('电影', 14),
 ('没', 14),
 ('奥创', 12),
 ('好看', 11),
 ('漫威', 10),
 ('英雄', 10),
 ('觉得', 9),
 ('.', 9),
 ('剧情', 9),
 ('居然', 8),
 ('真的', 8),
 ('超级', 8),
 ('反派', 8),
 ('~', 8),
 ('感觉', 7),
 ('第一部', 7),
 ('太', 7),
 ('喜欢', 6),
 (',', 6),
 ('动作', 5),
 ('联盟', 5),
 ('感情', 5),
 ('弱', 5),
 ('爆', 5),
 ('故事', 5),
 ('角色', 5),
 ('说', 5),
 ('睡着', 5),
 ('中', 5),
 ('打斗', 5),
 ('复联', 5),
 ('有点', 5),
 ('-', 5),
 ('黑寡妇', 5),
 ('系列', 5),
 ('一个', 4),
 ('冗长', 4),
 ('场面', 4),
 ('美国', 4),
 ('拍', 4),
 ('问题', 4),
 ('死', 4),
 ('期待', 4),
 ('复仇者', 4),
 ('片子', 4),
 ('一部', 4),
 ('浩克', 4),
 ('钢铁', 4),
 ('侠', 4),
 ('已经', 4),
 ('终于', 4),
 ('fuck', 4),
 ('大片', 4),
 ('was', 4),
 ('知道', 3),
 ('一直', 3),
 ('会', 3),
 ('依然', 3),
 ('生硬', 3),
 ('翻译', 3),
 ('漫画', 3),
 ('真是', 3),
 ('能力', 3),
 ('完全', 3),
 ('明白', 3),
 ('里', 3),
 ('差', 3),
 ('讲', 3),
 ('更', 3),
 ('节奏', 3),
 ('竟然', 3),
 ('快', 3),
 ('银', 3),
 ('真心', 3),
 ('还好', 3),
 ('一起', 3),
 ('绿巨人', 3),
 ('演员', 3),
 ('鹰眼', 3),
 ('戏份', 3),
 ('完', 3),

In [119]:
data['coment_processed'].head()

0                                         [奥创, 知道, 韩国]
1    [一个, 没有, 值得,  , 第二部, 冗长, 一直, 会, 觉得, 剩, 动作, 蛮, ...
2                               [奥创, 弱, 爆, 弱, 爆, 弱, 爆]
3    [不会, 好看, 喜欢, 漫威, 电影, 场面, 宏大, 故事, 冷笑, 话, 依然, 不少...
4                               [说, 奥创, 台北, 买, ......]
Name: coment_processed, dtype: object

In [120]:
#把数据分为训练集和测试集，comments_train(list)保存用于训练的文本你，comment_test(list)保存用于测试的文本
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(data,test_size=0.2)

x_train,x_test = train_data['coment_processed'],test_data['coment_processed']
y_train,y_test = train_data['Star'],test_data['Star']



In [121]:
#把训练文本和测试文本转换成tf-idf向量。
#使用sklearn的featyre_extarction.text.text.TfidfTransformer模块
#注意fit_transform和transform的区别。
#注意结果是否为稀疏矩阵
comments_train_concat = x_train.apply(lambda x:''.join(x))
comments_test_concat = x_test.apply(lambda x:''.join(x))

vectorizer = CountVectorizer()
trans = TfidfTransformer()

word_count_train = vectorizer.fit_transform(comments_train_concat)
tfidf_train = trans.fit_transform(word_count_train)

word_count_train = vectorizer.transform(comments_test_concat)
tfidf_test = trans.transform(word_count_train)

print(tfidf_train.shape,tfidf_test.shape)

(80, 101) (20, 101)


In [122]:
tfidf_train

<80x101 sparse matrix of type '<class 'numpy.float64'>'
	with 101 stored elements in Compressed Sparse Row format>

In [123]:
#把文本转换成word2vec向量
model = KeyedVectors.load_word2vec_format('E:\Game\pythonProject\douban/sgns.zhihu.word')

In [124]:
#预训练词向量使用举例
model['今天']

array([-3.51068e-01,  2.57389e-01, -1.46752e-01, -4.45400e-03,
       -1.04235e-01,  3.72475e-01, -4.29349e-01, -2.80470e-02,
        1.56651e-01, -1.27600e-01, -1.68833e-01, -2.91350e-02,
        4.57850e-02, -3.53735e-01,  1.61205e-01, -1.82645e-01,
       -1.35340e-02, -2.42591e-01, -1.33356e-01, -1.31012e-01,
       -9.29500e-02, -1.70479e-01, -2.54004e-01, -1.20530e-01,
       -1.33690e-01,  7.84360e-02, -1.46603e-01, -2.77378e-01,
       -1.36723e-01,  9.29070e-02, -4.00197e-01,  2.80726e-01,
       -1.73282e-01,  8.56630e-02,  2.37251e-01,  6.24290e-02,
       -1.57132e-01,  2.15685e-01,  9.54770e-02,  1.09896e-01,
       -2.05394e-01, -3.37900e-03, -2.77480e-02,  8.16580e-02,
        9.65290e-02,  1.23188e-01,  9.55090e-02, -2.31017e-01,
       -8.59590e-02, -2.21634e-01, -1.37885e-01, -1.84790e-01,
       -2.40127e-01, -2.79150e-01, -4.56200e-03,  1.04099e-01,
        3.20523e-01, -6.77270e-02,  1.95719e-01,  4.06145e-01,
       -2.98546e-01, -1.67750e-02,  2.74917e-01, -9.023

In [125]:
vocabulary = model.key_to_index
len(vocabulary)

259869

In [126]:
vocabulary

{'，': 0,
 '的': 1,
 '。': 2,
 '了': 3,
 '和': 4,
 '是': 5,
 '、': 6,
 '一个': 7,
 '我': 8,
 '有': 9,
 '（': 10,
 '：': 11,
 '不': 12,
 '都': 13,
 '在': 14,
 '上': 15,
 '他': 16,
 '个': 17,
 '人': 18,
 '被': 19,
 '“': 20,
 '）': 21,
 '要': 22,
 '就是': 23,
 '去': 24,
 '？': 25,
 '”': 26,
 '也': 27,
 '可以': 28,
 '！': 29,
 '会': 30,
 '没有': 31,
 '这': 32,
 '对': 33,
 '很': 34,
 '我们': 35,
 '还': 36,
 '说': 37,
 '把': 38,
 '年': 39,
 '自己': 40,
 '一': 41,
 '就': 42,
 '.': 43,
 '到': 44,
 '这个': 45,
 '《': 46,
 ',': 47,
 '能': 48,
 '》': 49,
 '中': 50,
 '里': 51,
 '为': 52,
 '与': 53,
 '不是': 54,
 '她': 55,
 '来': 56,
 '多': 57,
 '给': 58,
 '时候': 59,
 '你': 60,
 '/': 61,
 '着': 62,
 '用': 63,
 '还是': 64,
 '而': 65,
 '后': 66,
 '看': 67,
 '大': 68,
 '从': 69,
 '让': 70,
 '又': 71,
 '但': 72,
 '什么': 73,
 '因为': 74,
 '过': 75,
 '做': 76,
 '没': 77,
 '地': 78,
 '中国': 79,
 '好': 80,
 '啊': 81,
 '他们': 82,
 '但是': 83,
 '两': 84,
 '次': 85,
 '时': 86,
 '想': 87,
 '吧': 88,
 '；': 89,
 '觉得': 90,
 '那': 91,
 '(': 92,
 '小': 93,
 '知道': 94,
 '几': 95,
 '已经': 96,
 '等': 97,
 '之': 98,
 '

In [127]:
#测试     shape[0]向量维度
model['鲁迅'].shape[0]

300

In [128]:
#对于每个句子，生产句子的向量：包含在句子中的所有单词的向量做平均
vec_len = model['鲁迅'].shape[0]
def comm_vec(c):
    vec_com = np.zeros(vec_len)   #初始化词向量长度的0向量
    coun = 0
    for w in c:
        if w in model:
            vec_com += model[w]
            coun += 1
    return vec_com/coun
word2vec_train = np.vstack(x_train.progress_apply(comm_vec))
word2vec_test = np.vstack(x_test.progress_apply(comm_vec))

print(word2vec_train.shape,word2vec_test.shape)

Untitled-1: 100%|██████████| 20/20 [00:00<00:00, 6707.13it/s]

(80, 300) (20, 300)


In [129]:
print(tfidf_train.shape,tfidf_test.shape)
print(word2vec_train.shape,word2vec_test.shape)

(80, 101) (20, 101)
(80, 300) (20, 300)


In [130]:
#训练模型以及评估，训练逻辑回归模型
#导入逻辑回归的包
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [131]:
#使用tf-idf，结合逻辑回归训练模型
from sklearn.model_selection import ParameterSampler


lr = LogisticRegression()

parameters = {'C':[0.001,0.01,0.1,1,10,100]}
lr_grid_search = RandomizedSearchCV(estimator = lr,
                                    param_distributions = parameters,
                                    cv = 5,
                                    n_jobs = -1)

lr_gs = lr_grid_search.fit(tfidf_train,y_train)
lr_best_score = lr_grid_search.best_score_
lr_bes_param = lr_grid_search.best_params_

d:\Users\86183\anaconda3\envs\dl\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [132]:
print('BEST SCORE:{} %'.format(lr_best_score))
print('BEST Parameters:{} %'.format(lr_bes_param))

BEST SCORE:0.8 %
BEST Parameters:{'C': 0.001} %


In [134]:
lr_best = LogisticRegression(penalty='l2')
lr_best.fit(tfidf_train,y_train)
tf_idf_y_pred = lr_best.predict(tfidf_test)
print('Tf-IDF LR test accuracy %s' % metrics.accuracy_score(y_test,tf_idf_y_pred))
#逻辑回归模型在测试集上的F1——Score
print('Tf-IDF LR test F1_score %s' % metrics.f1_score(y_test,tf_idf_y_pred,average="macro"))


Tf-IDF LR test accuracy 0.7
Tf-IDF LR test F1_score 0.4117647058823529


In [135]:
#使用word2vec，结合逻辑回归训练模型
np.isinf(word2vec_train).any()


False

In [136]:
np.isnan(word2vec_train).any()

True

In [137]:
np.average(word2vec_train)

nan

In [138]:
word2vec_train[np.isnan(word2vec_train)] = 0
word2vec_test[np.isnan(word2vec_test)] = 0


In [139]:
#村联逻辑回归模型，用gridsearchCV做交叉验证，选择最好的超参数
from sklearn.linear_model import Log
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()

parameters = {'C':[0.0001,0.001,0.01,0.1,1,10,100],'solver':['liblinear','lbfgs','sag','saga']}

lr_grid_search = GridSearchCV(estimator = lr,
param_grid = parameters,
cv = 5,
n_jobs = -1
)
lr_gs = lr_grid_search.fit(tfidf_train,y_train)
lr_best_score = lr_grid_search.best_score_
lr_bes_param = lr_grid_search.best_params_
print('BEST SCORE:{} %'.format(lr_best_score))
print('BEST Parameters:{} %'.format(lr_bes_param))

BEST SCORE:0.8 %
BEST Parameters:{'C': 0.0001, 'solver': 'liblinear'} %


In [140]:
lr_best = LogisticRegression(penalty='l1',C=100,solver='liblinear')
lr_best.fit(word2vec_train,y_train)
word2vec_y_pred = lr_best.predict(word2vec_test)
print('word2vec LR test accuracy %s' % metrics.accuracy_score(y_test,word2vec_y_pred))
#逻辑回归模型在测试集上的F1——Score
print('word2vec LR test F1_score %s' % metrics.f1_score(y_test,word2vec_y_pred,average="macro"))


word2vec LR test accuracy 0.65
word2vec LR test F1_score 0.3939393939393939
